In [1]:
import pandas as pd

In [26]:
# Load data
customer_feedback = pd.read_parquet('C:/MSIB_DE/Data_Sinta-Ayu-Rismawati/CodeCompetence2/DataIngestionResult/customer_feedback.parquet')
customers = pd.read_parquet('C:/MSIB_DE/Data_Sinta-Ayu-Rismawati/CodeCompetence2/DataIngestionResult/customers.parquet')
events = pd.read_parquet('C:/MSIB_DE/Data_Sinta-Ayu-Rismawati/CodeCompetence2/DataIngestionResult/events.parquet')
tickets = pd.read_parquet('C:/MSIB_DE/Data_Sinta-Ayu-Rismawati/CodeCompetence2/DataIngestionResult/tickets.parquet')
transactions = pd.read_parquet('C:/MSIB_DE/Data_Sinta-Ayu-Rismawati/CodeCompetence2/DataIngestionResult/transactions.parquet')

# Transform 
combine1 = pd.merge(transactions, tickets, on=['ticket_id'], suffixes=('_transaction', '_tickets'))
combine2 = pd.merge(combine1, events, on=['event_id'], suffixes=('_ticket', '_event'))
combine2 = combine2.rename(columns={'date_ticket' : 'transaction_date'})
combine3 = pd.merge(transactions, customer_feedback, on=['transaction_id'])
combine4 = pd.merge(tickets, combine3, on=['ticket_id'])
combine5 = pd.merge(combine4, events, on=['event_id'])

# ticket_sold_per_event
ticket_sold_per_event = combine2.groupby(['event_id'], as_index=False).agg({
    'quantity' : 'sum',
    'event_id' : 'first',
    'name' : 'first',
    'location' : 'first',
    'category_event' : 'first'
})
ticket_sold_per_event = ticket_sold_per_event[['event_id', 'name', 'location', 'category_event', 'quantity']]
ticket_sold_per_event = ticket_sold_per_event.rename(columns={'quantity' : 'sold_quantity'})
ticket_sold_per_event.to_parquet('C:/MSIB_DE/Data_Sinta-Ayu-Rismawati/CodeCompetence2/DataForDW/ticket_sold_per_event.parquet')

# revenue_per_event
revenue_per_event = combine2.groupby(['event_id'], as_index=False).agg({
    'event_id' : 'first',
    'name' : 'first',
    'location' : 'first',
    'category_event' : 'first',
    'total_amount' : 'sum'
})
revenue_per_event = revenue_per_event[['event_id', 'name', 'location', 'category_event', 'total_amount']]
revenue_per_event.to_parquet('C:/MSIB_DE/Data_Sinta-Ayu-Rismawati/CodeCompetence2/DataForDW/revenue_per_event.parquet')

# feedback analysis
feedback_analysis = combine5.groupby(['event_id'], as_index=False).agg({
    'event_id' : 'first',
    'name' : 'first',
    'rating' : 'mean'
})
feedback_analysis.to_parquet('C:/MSIB_DE/Data_Sinta-Ayu-Rismawati/CodeCompetence2/DataForDW/feedback_analysis.parquet')

In [25]:
feedback_analysis

,event_id,name,rating
0,1,Ruteng Coffee and Music Festival,4.5
1,2,Jember Fashion Carnival,4.0
2,3,Sumarak Salingka Danau,3.5
3,4,Festival Pesona Danau Toba,3.5
4,5,Festival Budaya Tenun Ikat Flores,3.5
